# General Elections 2006-2020

Note about the data: 2002-2004 are missing houshoeld income information. 
2006-2010: delete the "OUT" columns

## Install Packages

In [11]:
#Check my working directory
!pwd

#Import sys to install packages in virtual env (using virtual env for the kernel)
import sys

#Install openpyxl with the sys method to locally install in my virtual env
sys.path.append('./lib/python3.7/site-packages')
!{sys.executable} -m pip install openpyxl
!{sys.executable} -m pip install numpy

#Import other packages for analysis
import pandas as pd
from openpyxl import Workbook
from openpyxl import load_workbook
import numpy as np

/Users/amy/Code/injustice_watch/analysis


## Define Functions

Converts an excel file to a pandas dataframe.

In [12]:
#Pass a string pathname
#Returns a pandas df
def excel_to_df(pathname): 
    wb = load_workbook(pathname)
    ws = wb.active
    data = ws.values
    columns = next(data)[0:] #Gets the first line in the file as a header line
    df = pd.DataFrame(data, columns=columns)
    
    return df

Removes ballot measures from the data.

In [13]:
#Pass a df that contains all the ballot measures
#Returns a cleaned df that deletes all rows with the ballot measures
def remove_ballot_measures(ballot_measures_df,df):
    clean_df = df[~df.RACE.isin(ballot_measures_df.RACE)]
    
    return clean_df

Finds and tags retention races.

In [14]:
#Pass a df
#Returns the df with an added row called 'Retention'
def tag_retention(df):
    def retention(a):
        if 'No' in a:
            return 'Retention'
        elif 'Yes' in a:
            return 'Retention'
        else:
            return 'Not Retention'
    
    df['Retention'] = df['CANDIDATE'].apply(lambda x: retention(x))
    
    return df

Finds and tags subcircuit races.

In [57]:
def tag_subcircuit(df):
    def subcircuit(a):
        if 'Subcircuit' in a:
            return 'Subcircuit'
        elif 'Sub' in a: #accounts for 2006 notation
            return 'Subcircuit'
        else:
            return 'Not Subcircuit'
   
    df['Subcircuit'] = df['RACE'].apply(lambda x: subcircuit(x))
    
    return df

Creates all_races_df which is an aggregate by judicial race.

In [16]:
#Pass a clean_df
#Returns a df of aggregated by judicial race by sum of votes
def create_all_races(clean_df):
    clean_df['key'] = clean_df['RACE'] + clean_df['WARD'].map(str)
    all_races_df = clean_df.groupby(by=['key']).agg({'WARD': 'first','REGISTERED VOTERS': 'first',
                                                     'BALLOTS CAST': 'first','VOTES': 'sum',
                                                     'Retention': 'first','Subcircuit':'first'})
    return all_races_df

Creates ward_df which is an aggregate of all races by ward.

In [17]:
#Pass all_races_df
#Returns ward_df for all judicial races that aggregates votes by sum and includes count of races for each ward
def create_ward_df(all_races_df):
    ward_df = all_races_df.groupby(by=['WARD']).agg({'REGISTERED VOTERS': 'first','BALLOTS CAST': 'first',
                                                     'WARD':'count','VOTES': 'sum'})
    #Rename column for vote and decapitalize
    ward_df = ward_df.rename(columns={'REGISTERED VOTERS': 'Registered Voters', 'BALLOTS CAST': 'Ballots Cast', 
                                      'WARD': 'Number of Races', 'VOTES': 'Sum of Votes'})
    
    #Ward column becomes an index column in the groupby; this resets it as it's own column to merge on
    ward_df.reset_index(inplace=True)
    
    return ward_df

Similar to ward_df, but for retention races only.

In [18]:
#Pass retention_races_df
#Returns retention_ward_df
def create_retention_ward_df(retention_races_df):
    retention_ward_df = retention_races_df.groupby(by=['WARD']).agg({'REGISTERED VOTERS': 'first',
                                                                     'BALLOTS CAST': 'first','WARD':'count',
                                                                     'VOTES': 'sum'})
    retention_ward_df = retention_ward_df.rename(columns={'REGISTERED VOTERS': 'Registered Voters',
                                                          'BALLOTS CAST': 'Ballots Cast', 
                                                          'WARD': 'Number of Retention Races', 
                                                          'VOTES': 'Sum of Retention Votes'})
    retention_ward_df.reset_index(inplace=True)
    
    return retention_ward_df

Similar to ward_df, but for vacancy races only.

In [19]:
def create_vacancy_ward_df(vacancy_races_df):
    vacancy_ward_df = vacancy_races_df.groupby(by=['WARD']).agg({'REGISTERED VOTERS': 'first',
                                                                 'BALLOTS CAST': 'first','WARD':'count',
                                                                 'VOTES': 'sum'})
    vacancy_ward_df = vacancy_ward_df.rename(columns={'REGISTERED VOTERS': 'Registered Voters', 
                                                      'BALLOTS CAST': 'Ballots Cast', 'WARD': 'Number of Vacancy Races', 
                                                      'VOTES': 'Sum of Vacancy Votes'})
    vacancy_ward_df.reset_index(inplace=True)
    
    return vacancy_ward_df

Do the same for subcircuits and circuits.

In [20]:
def create_subcircuit_ward_df(subcircuit_races_df):
    subcircuit_ward_df = subcircuit_races_df.groupby(by=['WARD']).agg({'REGISTERED VOTERS': 'first',
                                                                       'BALLOTS CAST': 'first','WARD':'count',
                                                                       'VOTES': 'sum'})
    subcircuit_ward_df = subcircuit_ward_df.rename(columns={'REGISTERED VOTERS': 'Registered Voters', 
                                                            'BALLOTS CAST': 'Ballots Cast', 
                                                            'WARD': 'Number of Subcircuit Races', 
                                                            'VOTES': 'Sum of Subcircuit Votes'})
    
    subcircuit_ward_df.reset_index(inplace=True)
    
    return subcircuit_ward_df

In [29]:
def create_circuit_ward_df(circuit_races_df):
    circuit_ward_df = circuit_races_df.groupby(by=['WARD']).agg({'REGISTERED VOTERS': 'first',
                                                                       'BALLOTS CAST': 'first','WARD':'count',
                                                                       'VOTES': 'sum'})
    circuit_ward_df = circuit_ward_df.rename(columns={'REGISTERED VOTERS': 'Registered Voters', 
                                                            'BALLOTS CAST': 'Ballots Cast', 
                                                            'WARD': 'Number of Circuit Races', 
                                                            'VOTES': 'Sum of Circuit Votes'})
    
    circuit_ward_df.reset_index(inplace=True)
    
    return circuit_ward_df

Prepare demographics data (clean and join to final_df)

In [58]:
#Pass a final_df and demo_df
#Returns a new joined_df with the cleaned demos joined to final
def prep_demos(final_df,demo_df):
    #Add calculated VAP column
    demo_df['Citizens Over 18'] = demo_df['Native Born, Male, Over 18'] + demo_df['Naturalized U.S. Citizen, Male, Over 18'] + demo_df['Native Born, female, Over 18'] + demo_df['Naturalized U.S. Citizen, Female, Over 18']
    
    #Add calculated race columns
    demo_df['Pct White'] = demo_df['Not Hispanic or Latino Origin, Whites']/demo_df['Total Population']
    demo_df['Pct Black'] = demo_df['Not Hispanic or Latino Origin, Blacks']/demo_df['Total Population']
    demo_df['Pct Latino'] = demo_df['Hispanic or Latino Origin']/demo_df['Total Population']
    demo_df['Pct Asian'] = demo_df['Not Hispanic or Latino Origin, Asians']/demo_df['Total Population']
    
    #Drop all absolute race columms, disaggregated citizen, income and household stuff, and number of tracts
    demo_df = demo_df.drop(['Native Born, Male, Over 18','Naturalized U.S. Citizen, Male, Over 18',
                            'Native Born, female, Over 18','Naturalized U.S. Citizen, Female, Over 18',
                           'number of tracts','Not Hispanic or Latino Origin, Whites','Not Hispanic or Latino Origin, Blacks',
                           'Hispanic or Latino Origin','Not Hispanic or Latino Origin, Asians',
                            'Aggregate household income in the past 12 months','Total households',
                           'mean household income'],axis=1)
    
    #Join demographics with final
    final_df['WARD'].astype(int)
    demo_df['ward'].astype(int)
    joined_df = pd.merge(final_df, demo_df, how='inner', left_on = 'WARD', right_on = 'ward')
    
    return joined_df

Max votes is the maximum number of votes if 100% of people who cast a ballot in an election voted in 100% of judicial races.

Theoretically possible votes is the maximium number of votes if 100% of registered voters voted in 100% of judicial races.

Judicial participation rate which is the inverse of the roll-off rate.

Turnout is ballots cast over total registered voters.

In [23]:
#Pass joined_df
#Returns a full_df which is joined_df with additional calculated columns
def add_calculations(joined_df):
    #Make a deep copy of joined_df so as to not modify that
    full_df = joined_df.copy()
    
    #Max votes
    full_df['Max Votes ALL'] = full_df['Ballots Cast']*full_df['Number of Races']
    full_df['Max Votes RETENTION'] = full_df['Ballots Cast']*full_df['Number of Retention Races']
    full_df['Max Votes VACANCY'] = full_df['Ballots Cast']*full_df['Number of Vacancy Races']
    full_df['Max Votes SUBCIRCUIT'] = full_df['Ballots Cast']*full_df['Number of Subcircuit Races']
    full_df['Max Votes CIRCUIT'] = full_df['Ballots Cast']*full_df['Number of Circuit Races']
    
    #Theoretical votes
    full_df['Theoretical Votes ALL'] = full_df['Citizens Over 18']*full_df['Number of Races']
    full_df['Theoretical Votes RETENTION'] = full_df['Citizens Over 18']*full_df['Number of Retention Races']
    full_df['Theoretical Votes VACANCY'] = full_df['Citizens Over 18']*full_df['Number of Vacancy Races']
    full_df['Theoretical Votes SUBCIRCUIT'] = full_df['Citizens Over 18']*full_df['Number of Subcircuit Races']
    full_df['Theoretical Votes CIRCUIT'] = full_df['Citizens Over 18']*full_df['Number of Circuit Races']
    
    #Judicial participation
    full_df['Judicial Participation ALL'] = full_df['Sum of Votes']/full_df['Max Votes ALL']
    full_df['Judicial Participation RETENTION'] = full_df['Sum of Retention Votes']/full_df['Max Votes RETENTION']
    full_df['Judicial Participation VACANCY'] = full_df['Sum of Vacancy Votes']/full_df['Max Votes VACANCY']
    full_df['Judicial Participation SUBCIRCUIT'] = full_df['Sum of Subcircuit Votes']/full_df['Max Votes SUBCIRCUIT']
    full_df['Judicial Participation CIRCUIT'] = full_df['Sum of Circuit Votes']/full_df['Max Votes CIRCUIT']
    
    #Turnout for judicial
    full_df['Judicial Turnout ALL'] = full_df['Sum of Votes']/full_df['Theoretical Votes ALL']
    full_df['Judicial Turnout RETENTION'] = full_df['Sum of Retention Votes']/full_df['Theoretical Votes RETENTION']
    full_df['Judicial Turnout VACANCY'] = full_df['Sum of Vacancy Votes']/full_df['Theoretical Votes VACANCY']
    full_df['Judicial Turnout SUBCIRCUIT'] = full_df['Sum of Subcircuit Votes']/full_df['Theoretical Votes SUBCIRCUIT']
    full_df['Judicial Turnout CIRCUIT'] = full_df['Sum of Circuit Votes']/full_df['Theoretical Votes CIRCUIT']
    
    #Overall Turnout
    full_df['Overall Turnout'] = full_df['Ballots Cast']/full_df['Citizens Over 18']
    
    return full_df

## Loop through all years

Define pathnames to access the data.

In [33]:
judicial_list = ['/Users/amy/Code/injustice_watch/Judicial General Data/judicial_general_2006.xlsx',
                '/Users/amy/Code/injustice_watch/Judicial General Data/judicial_general_2008.xlsx',
                '/Users/amy/Code/injustice_watch/Judicial General Data/judicial_general_2010.xlsx',
                '/Users/amy/Code/injustice_watch/Judicial General Data/judicial_general_2012.xlsx',
                '/Users/amy/Code/injustice_watch/Judicial General Data/judicial_general_2014.xlsx',
                '/Users/amy/Code/injustice_watch/Judicial General Data/judicial_general_2016.xlsx',
                '/Users/amy/Code/injustice_watch/Judicial General Data/judicial_general_2018.xlsx',
                '/Users/amy/Code/injustice_watch/Judicial General Data/judicial_general_2020.xlsx']

demographics_list = ['/Users/amy/Code/injustice_watch/Judicial General Data/Demographics VAP/ward_2006_general.csv',
                    '/Users/amy/Code/injustice_watch/Judicial General Data/Demographics VAP/ward_2008_general.csv',
                    '/Users/amy/Code/injustice_watch/Judicial General Data/Demographics VAP/ward_2010_general.csv',
                    '/Users/amy/Code/injustice_watch/Judicial General Data/Demographics VAP/ward_2012_general.csv',
                    '/Users/amy/Code/injustice_watch/Judicial General Data/Demographics VAP/ward_2014_general.csv',
                    '/Users/amy/Code/injustice_watch/Judicial General Data/Demographics VAP/ward_2016_general.csv',
                    '/Users/amy/Code/injustice_watch/Judicial General Data/Demographics VAP/ward_2018_general.csv',
                    '/Users/amy/Code/injustice_watch/Judicial General Data/Demographics VAP/ward_2020_general.csv']

Loop through all the years of data and return a dictionary of the year and df.

In [75]:
#Create two dictionary to store results in
full_df_dict = {}
short_df_dict = {}

#Create a counter to index the position of the items in judicial_list and demographics_list
index = 0

#Create a list of years to index for the file names
years = ['2006','2008','2010','2012','2014','2016','2018','2020']

#Load ballot measures df
ballot_measures_df = excel_to_df('/Users/amy/Code/injustice_watch/Judicial General Data/ballot_measures.xlsx')

#Loop through years and call on functions above
for pathname in judicial_list:
    print(index)
    
    #Load data
    df = excel_to_df(pathname)
    
    #Clean data
    df = tag_retention(df)
    df = tag_subcircuit(df)
    clean_df = remove_ballot_measures(ballot_measures_df,df)
    
    #Create judicial races dataframes
    all_races_df = create_all_races(clean_df)
    retention_races_df = all_races_df[all_races_df['Retention'] == 'Retention']
    vacancy_races_df = all_races_df[all_races_df['Retention'] == 'Not Retention']
    subcircuit_races_df = all_races_df[all_races_df['Subcircuit'] == 'Subcircuit']
    circuit_races_df = all_races_df[all_races_df['Subcircuit'] == 'Not Subcircuit']
    
    #Create ward dataframes
    ward_df = create_ward_df(all_races_df)
    retention_ward_df = create_retention_ward_df(retention_races_df)
    vacancy_ward_df = create_vacancy_ward_df(vacancy_races_df)
    subcircuit_ward_df = create_subcircuit_ward_df(subcircuit_races_df)
    circuit_ward_df = create_circuit_ward_df(circuit_races_df)
    
    #print('subcircuit ward')
    #print(subcircuit_ward_df.head())
    
    #Join just the avg votes by retention/vacancy and subcircuit/circuit to the ward_df to create an analysis_df
    merge1 = pd.merge(ward_df, retention_ward_df.drop(['Registered Voters','Ballots Cast'],axis=1), how='outer', left_on = 'WARD', right_on = 'WARD')
    merge2 = pd.merge(merge1, vacancy_ward_df.drop(['Registered Voters','Ballots Cast'],axis=1), how='outer', left_on = 'WARD', right_on = 'WARD')
    merge3 = pd.merge(merge2, subcircuit_ward_df.drop(['Registered Voters','Ballots Cast'],axis=1), how='outer', left_on = 'WARD', right_on = 'WARD')
    final_df = pd.merge(merge3, circuit_ward_df.drop(['Registered Voters','Ballots Cast'],axis=1), how='outer', left_on = 'WARD', right_on = 'WARD')
    
    #print('final df')
    #print(final_df.head())
    
    #Load demographics data
    demo_df = pd.read_csv(demographics_list[index])
    
    #Clean and join demographics data
    joined_df = prep_demos(final_df,demo_df)
    
    #Add calculations
    full_df = add_calculations(joined_df)
    
    #Replace NaN's from merges
    full_df = full_df.fillna('')
    
    #Create a short version with just demos and calculated columns by ward and not max/theoretical
    short_df = full_df.copy()
    short_df = short_df.drop(['ward','Theoretical Votes ALL',
                             'Theoretical Votes RETENTION','Theoretical Votes VACANCY',
                             'Theoretical Votes SUBCIRCUIT','Theoretical Votes CIRCUIT',
                             'Max Votes ALL','Max Votes RETENTION','Max Votes VACANCY','Max Votes SUBCIRCUIT',
                             'Max Votes CIRCUIT'],axis=1)
    
    #Add this year's dataframe to the dictionary with the year as the key
    current_year = years[index]
    print(current_year)
    full_df_dict[current_year] = full_df
    short_df_dict[current_year] = short_df
    
    #Update counter
    index+=1

0


<ipython-input-16-3bfd10a83fce>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_df['key'] = clean_df['RACE'] + clean_df['WARD'].map(str)


2006
1
2008
2
2010
3
2012
4
2014
5
2016
6
2018
7
2020


In [72]:
#Inspect
test = short_df_dict['2006']
test

,WARD,Total Population,Citizens Over 18,Pct White,Pct Black,Pct Latino,Pct Asian,Judicial Participation ALL,Judicial Participation RETENTION,Judicial Participation VACANCY,Judicial Participation SUBCIRCUIT,Judicial Participation CIRCUIT,Judicial Turnout ALL,Judicial Turnout RETENTION,Judicial Turnout VACANCY,Judicial Turnout SUBCIRCUIT,Judicial Turnout CIRCUIT,Overall Turnout
0,1.0,54613,39401,0.533261,0.056433,0.358907,0.034607,0.494479,0.474061,0.615288,0.360826,0.501247,0.143220,0.137306,0.178210,0.104509,0.145180,0.289637
1,2.0,56360,42342,0.376171,0.419163,0.057488,0.123510,0.541319,0.524089,0.663656,0.227799,0.549257,0.194349,0.188163,0.238272,0.0817864,0.197199,0.359029
2,3.0,36741,25171,0.099943,0.788846,0.064587,0.038622,0.547027,0.514966,0.799959,0.699534,0.545097,0.223714,0.210602,0.327153,0.286083,0.222924,0.408963
3,4.0,41945,29557,0.148313,0.742091,0.027608,0.062582,0.602602,0.576084,0.811794,0.811631,0.599956,0.319721,0.305652,0.430712,0.430626,0.318317,0.530568
4,5.0,41018,29482,0.162173,0.769272,0.023648,0.030621,0.568583,0.555514,0.652939,0.271223,0.579875,0.289499,0.282844,0.332449,0.138096,0.295248,0.509158
5,6.0,54150,39841,0.004931,0.974054,0.006759,0.002364,0.628696,0.608355,0.759990,0.523765,0.632681,0.289834,0.280456,0.350361,0.24146,0.291671,0.461008
6,7.0,38234,26551,0.014568,0.894727,0.077706,0.001334,0.633853,0.614543,0.758492,0.524273,0.638014,0.363085,0.352024,0.434481,0.300315,0.365469,0.572822
7,8.0,49613,37422,0.008163,0.974684,0.008022,0.003245,0.655037,0.635577,0.780643,0.557698,0.658734,0.346756,0.336454,0.413247,0.295227,0.348712,0.529368
8,9.0,46010,32014,0.017301,0.937557,0.039491,0.000935,0.614109,0.600430,0.702401,0.309441,0.625679,0.280736,0.274483,0.321098,0.141459,0.286025,0.457144
9,10.0,58321,31701,0.178786,0.205158,0.610586,0.001560,0.590208,0.564607,0.771977,0.746087,0.586262,0.193869,0.185460,0.253576,0.245071,0.192573,0.328475


## Export to csv as 'analysis df [YEAR] V2'

FULL DF: Loop through the dictionary and export to csv individually (b/c I'm not sure the best way to do this in a batch yet)

In [76]:
for key,value in full_df_dict.items():
    pathname = '/Users/amy/Code/injustice_watch/analysis/full df_' + key + '.csv'
    value.to_csv(pathname)

SHORT DF

In [77]:
for key,value in short_df_dict.items():
    pathname = '/Users/amy/Code/injustice_watch/analysis/short df_' + key + '.csv'
    value.to_csv(pathname)